In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# import MultiColumnLabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import xgboost as xgb
import pickle

In [5]:
dataset = pd.read_csv("garments_worker_productivity.csv")
dataset.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [6]:
dataset.shape

(1197, 15)

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  no_of_style_change     1197 non-null   int64  
 13  no_of_workers          1197 non-null   float64
 14  actual_productivity    1197 non-null   float64
dtypes: f

In [8]:
dataset['team'] = dataset['team'].astype('int32')
dataset['over_time'] = dataset['team'].astype('int32')
dataset['incentive'] = dataset['incentive'].astype('int32')
dataset['idle_men'] = dataset['idle_men'].astype('int32')
dataset['no_of_style_change'] = dataset['no_of_style_change'].astype('int32')

dataset['targeted_productivity'] = dataset['targeted_productivity'].astype('float32')
dataset['smv'] = dataset['smv'].astype('float32')
dataset['idle_time'] = dataset['idle_time'].astype('float32')
dataset['no_of_workers'] = dataset['targeted_productivity'].astype('float32')
dataset['actual_productivity'] = dataset['actual_productivity'].astype('float32')


In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int32  
 5   targeted_productivity  1197 non-null   float32
 6   smv                    1197 non-null   float32
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int32  
 9   incentive              1197 non-null   int32  
 10  idle_time              1197 non-null   float32
 11  idle_men               1197 non-null   int32  
 12  no_of_style_change     1197 non-null   int32  
 13  no_of_workers          1197 non-null   float32
 14  actual_productivity    1197 non-null   float32
dtypes: f

In [10]:
dataset.describe()

,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
count,1197.000000,1197.000000,1197.000000,691.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000
mean,6.426901,0.729632,15.062174,1190.465991,6.426901,38.210526,0.730159,0.369256,0.150376,0.729632,0.735091
std,3.463963,0.097891,10.943233,1837.455001,3.463963,160.182643,12.709723,3.268987,0.427848,0.097891,0.174488
min,1.000000,0.070000,2.900000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.233705
25%,3.000000,0.700000,3.940000,774.500000,3.000000,0.000000,0.000000,0.000000,0.000000,0.700000,0.650307
50%,6.000000,0.750000,15.260000,1039.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.773333
75%,9.000000,0.800000,24.260000,1252.500000,9.000000,50.000000,0.000000,0.000000,0.000000,0.800000,0.850253
max,12.000000,0.800000,54.560001,23122.000000,12.000000,3600.000000,300.000000,45.000000,2.000000,0.800000,1.120438


In [11]:
# Number of NULL values
dataset.isna().sum()

date                       0
quarter                    0
department                 0
day                        0
team                       0
targeted_productivity      0
smv                        0
wip                      506
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
actual_productivity        0
dtype: int64

In [12]:
# Drop null values
dataset.drop("wip",axis=1, inplace=True)

In [13]:
# Number of Non-NULL values
dataset.notna().sum()

date                     1197
quarter                  1197
department               1197
day                      1197
team                     1197
targeted_productivity    1197
smv                      1197
over_time                1197
incentive                1197
idle_time                1197
idle_men                 1197
no_of_style_change       1197
no_of_workers            1197
actual_productivity      1197
dtype: int64

In [14]:
# Making the two date formats uniform and converting them into respective months
dataset['date'] = pd.to_datetime(dataset['date'])
dataset['month'] = dataset['date'].dt.month_name()

In [15]:
# No need of date column
dataset.drop("date", axis=1, inplace=True)

In [16]:
dataset

,quarter,department,day,team,targeted_productivity,smv,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity,month
0,Quarter1,sweing,Thursday,8,0.80,26.16,8,98,0.0,0,0,0.80,0.940725,January
1,Quarter1,finishing,Thursday,1,0.75,3.94,1,0,0.0,0,0,0.75,0.886500,January
2,Quarter1,sweing,Thursday,11,0.80,11.41,11,50,0.0,0,0,0.80,0.800570,January
3,Quarter1,sweing,Thursday,12,0.80,11.41,12,50,0.0,0,0,0.80,0.800570,January
4,Quarter1,sweing,Thursday,6,0.80,25.90,6,50,0.0,0,0,0.80,0.800382,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Quarter2,finishing,Wednesday,10,0.75,2.90,10,0,0.0,0,0,0.75,0.628333,March
1193,Quarter2,finishing,Wednesday,8,0.70,3.90,8,0,0.0,0,0,0.70,0.625625,March
1194,Quarter2,finishing,Wednesday,7,0.65,3.90,7,0,0.0,0,0,0.65,0.625625,March
1195,Quarter2,finishing,Wednesday,9,0.75,2.90,9,0,0.0,0,0,0.75,0.505889,March


In [17]:
dataset['quarter'].value_counts()

quarter
Quarter1    360
Quarter2    335
Quarter4    248
Quarter3    210
Quarter5     44
Name: count, dtype: int64

In [18]:
dataset['department'].value_counts()

department
sweing        691
finishing     257
finishing     249
Name: count, dtype: int64

In [19]:
dataset['department'].unique()

array(['sweing', 'finishing ', 'finishing'], dtype=object)

In [20]:
# Cleaning and fixing dataset for the correct Department value
dataset['department'] = dataset['department'].replace('sweing', 'Sewing')
dataset['department'] = dataset['department'].replace('finishing ', 'Finishing')
dataset['department'] = dataset['department'].replace('finishing', 'Finishing')

In [21]:
dataset['department'].value_counts()

department
Sewing       691
Finishing    506
Name: count, dtype: int64

In [22]:
dataset['day'].value_counts()

day
Wednesday    208
Sunday       203
Tuesday      201
Thursday     199
Monday       199
Saturday     187
Name: count, dtype: int64

In [23]:
dataset['month'].unique()

array(['January', 'February', 'March'], dtype=object)

In [24]:
X = dataset.drop(['actual_productivity'],axis=1)
y = dataset['actual_productivity']

In [25]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.8,random_state=0)

In [26]:
X_train

,quarter,department,day,team,targeted_productivity,smv,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,month
798,Quarter3,Sewing,Monday,8,0.70,30.100000,8,27,2.0,10,2,0.70,February
114,Quarter1,Sewing,Wednesday,2,0.80,28.080000,2,63,0.0,0,0,0.80,January
803,Quarter3,Sewing,Tuesday,1,0.80,22.520000,1,113,0.0,0,0,0.80,February
508,Quarter5,Sewing,Thursday,1,0.75,22.940001,1,81,0.0,0,0,0.75,January
871,Quarter4,Finishing,Sunday,9,0.80,3.940000,9,0,0.0,0,0,0.80,February
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,Quarter1,Sewing,Tuesday,8,0.75,29.400000,8,56,0.0,0,0,0.75,March
763,Quarter2,Sewing,Saturday,3,0.50,30.100000,3,30,0.0,0,1,0.50,February
835,Quarter3,Sewing,Wednesday,9,0.70,18.790001,9,30,0.0,0,1,0.70,February
559,Quarter1,Sewing,Sunday,7,0.70,24.260000,7,0,0.0,0,0,0.70,February


In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

### Linear Regression

In [28]:
step1 = ColumnTransformer(transformers=[
    ('tnf1', OneHotEncoder(sparse_output=False , handle_unknown='ignore'), ['quarter','department']),
    ('tnf2', OrdinalEncoder(categories = [['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday']]), ['day']),
    ('tnf3', OrdinalEncoder(categories = [['January', 'February', 'March']]), ['month'])
], remainder='passthrough')

step2 = LinearRegression()

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)

print('Mean squared Error ',mean_squared_error(y_test, y_pred))
print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

Mean squared Error  1.0061010523846858
R2 score -32.18934449757514
MAE 0.20197803746088566


#### RandomForest

In [29]:
step1 = ColumnTransformer(transformers=[
    ('tnf1', OneHotEncoder(sparse_output=False ,handle_unknown='ignore'), ['quarter','department']),
    ('tnf2', OrdinalEncoder(categories = [['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday']]), ['day']),
    ('tnf3', OrdinalEncoder(categories = [['January', 'February', 'March']]), ['month'])
], remainder='passthrough')

step2 = RandomForestRegressor(n_estimators=100,
                              max_depth=15)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)

print('Mean squared Error ',mean_squared_error(y_test, y_pred))
print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

Mean squared Error  0.0210813663251011
R2 score 0.30456614891221034
MAE 0.09719722347540447


### XGBoost Model

In [30]:
step1 = ColumnTransformer(transformers=[
    ('tnf1', OneHotEncoder(sparse_output=False ,handle_unknown='ignore'), ['quarter','department']),
    ('tnf2', OrdinalEncoder(categories = [['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday']]), ['day']),
    ('tnf3', OrdinalEncoder(categories = [['January', 'February', 'March']]), ['month'])
], remainder='passthrough')

step2 = xgb.XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.1)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)

print('Mean squared Error ',mean_squared_error(y_test, y_pred))
print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

Mean squared Error  0.024784494
R2 score 0.18240707101787634
MAE 0.10378992


### Importing Model

In [33]:
pickle.dump(dataset,open('dataset.pkl','wb'))
pickle.dump(pipe,open('pipe.pkl','wb'))

In [32]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 958 entries, 700 to 29
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   quarter                958 non-null    object 
 1   department             958 non-null    object 
 2   day                    958 non-null    object 
 3   team                   958 non-null    int32  
 4   targeted_productivity  958 non-null    float32
 5   smv                    958 non-null    float32
 6   over_time              958 non-null    int32  
 7   incentive              958 non-null    int32  
 8   idle_time              958 non-null    float32
 9   idle_men               958 non-null    int32  
 10  no_of_style_change     958 non-null    int32  
 11  no_of_workers          958 non-null    float32
 12  month                  958 non-null    object 
dtypes: float32(4), int32(5), object(4)
memory usage: 71.1+ KB
